# We verify that MLP can generalize symbolic constraints
- Sum of 25
- Increasing
- Symmetric
- Odd/Even (weird?) or Dividable by something

Real is much easier.
For symbolic, it would not be much harder if internally it learns embeddings for each number.

In [115]:
import torch
from torch import nn
import numpy as np
from coins import generate_combinations
from itertools import product
from torch.utils.data import DataLoader

CONSTRAINT = 'sum_25'
#CONSTRAINT = 'increasing'

if CONSTRAINT == 'sum_25':
    combinations = np.asarray(generate_combinations(25, range(10), 5))
    combinations_set = set(map(tuple, combinations))
    uniform = np.asarray(list(product(range(10),range(10),range(10),range(10),range(10))))
    non_combinations = np.asarray([c for c in uniform if tuple(c) not in combinations_set])
    positive = combinations.astype(np.float32)
    negative = non_combinations.astype(np.float32)
    
    
def split(iterable, train_ratio=0.1, shuffle=True):
    if shuffle:
        np.random.shuffle(iterable)
    n = int(len(iterable)*train_ratio)
    train = iterable[:n]
    test = iterable[n:]
    return train, test

train_positive, test_positive = split(positive)
train_negative, test_negative = split(negative)

BATCH = 32

def make_infinite(iterable):
    while True:
        for i in iterable:
            yield i
            
            
train_positive_iter = make_infinite(DataLoader(train_positive, batch_size=BATCH, shuffle=True))
test_positive_iter = make_infinite(DataLoader(test_positive, batch_size=BATCH, shuffle=True))
train_negative_iter = make_infinite(DataLoader(train_negative, batch_size=BATCH, shuffle=True))
test_negative_iter = make_infinite(DataLoader(test_negative, batch_size=BATCH, shuffle=True))

print 'Positives', len(train_positive)
print 'Negatives', len(train_negative)

Positives 563
Negatives 9436


In [114]:
dims = 100
model = nn.Sequential(
    nn.Linear(5, dims),
    nn.ReLU(True),
    nn.Linear(dims, dims),
    nn.ReLU(True),
    nn.Linear(dims, 2),
    nn.LogSoftmax()
)

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())
losses = []
test_losses = []
test_accuracies = []

def get_loss(model, positive, negative):
    positive_pred = model(positive)
    negative_pred = model(negative)    
    positive_target = torch.ones(len(positive), dtype=torch.long)
    negative_target = torch.zeros(len(negative), dtype=torch.long)
    
    positive_loss = criterion(positive_pred, positive_target)
    negative_loss = criterion(negative_pred, negative_target)
    
    positive_accuracy = (positive_pred.argmax(1) == positive_target).float().mean()
    negative_accuracy = (negative_pred.argmax(1) == negative_target).float().mean()
    
    loss = 0.5 * (positive_loss + negative_loss)
    accuracy = 0.5 * (positive_accuracy + negative_accuracy)
    
    # Accuracy
    return loss, accuracy


for iteration in xrange(10000):
    # train
    positive = train_positive_iter.next()
    negative = train_negative_iter.next()
    loss, accuracy = get_loss(model, positive, negative)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # test
    test_positive = test_positive_iter.next().float()
    test_negative = test_negative_iter.next().float()    
    test_loss, test_accuracy = get_loss(model, test_positive, test_negative)
  
    losses.append(loss.item())
    test_losses.append(test_loss.item())
    test_accuracies.append(test_accuracy.item())
    if iteration % 500 == 0:
        print 'Iteration', iteration
        print 'Train', np.mean(losses[-min(len(losses), 50):])
        print 'Test ', np.mean(test_losses[-min(len(losses), 50):])
        print 'Test Accuracy ', np.mean(test_accuracies[-min(len(losses), 50):])

Iteration 0
Train 0.7520976662635803
Test  0.689124584197998
Test Accuracy  0.5625
Iteration 500
Train 0.493678143620491
Test  0.5139653617143631
Test Accuracy  0.7396875
Iteration 1000
Train 0.40479266464710234
Test  0.4165205144882202
Test Accuracy  0.8105208325386047
Iteration 1500
Train 0.2820704773068428
Test  0.2869487324357033
Test Accuracy  0.9009375
Iteration 2000
Train 0.17585505589842795
Test  0.18914292097091676
Test Accuracy  0.9396875
Iteration 2500
Train 0.14609171621501446
Test  0.1459859184920788
Test Accuracy  0.9525
Iteration 3000
Train 0.09876847863197327
Test  0.11125986143946648
Test Accuracy  0.9659375
Iteration 3500
Train 0.07955145843327045
Test  0.10135168120265008
Test Accuracy  0.9646875
Iteration 4000
Train 0.05866503246128559
Test  0.07112564716488123
Test Accuracy  0.9809375
Iteration 4500
Train 0.0628455414250493
Test  0.07323305584490299
Test Accuracy  0.976875
Iteration 5000
Train 0.05540789362043142
Test  0.07266487112268806
Test Accuracy  0.9784375
I

In [89]:
a = model(torch.Tensor([[5,6,5,5,5]]))
print a
a.argmax(1)

tensor([[-0.6838, -0.7026]])


tensor([ 0])

In [41]:
torch.long

torch.int64